<b><font size=20, color='#A020F0'>Class Notebook 7</font></b>

## Welcome to your seventh in-class jupyter notebook activity!
In this notebook you'll get more practice with xgcm

---

<b><font color='red'>Due Date: 18 November 2024 @ 9 am</font></b><br>

<b>How you will turn in this assignment</b><br> When you are ready to turn in your notebook, do the following:
1. Save your notebook and make sure your results are visible (cells have been executed)
2. Open a terminal and navigate to your local `aos573_completed_assignments` repository
3. Move your completed in-class notebook into your `Class_Notebooks` directory
4. `add` and `commit` your completed notebook
5. Finally, `push` your changes to your remote `aos573_completed_assignments` repository: `git push finished_work main` (you'll need to enter your username and personal access token)
6. <b><font color='red'>REMINDER: DO NOT TRACK OR PUSH THE DATA USED IN THE NOTEBOOK</font></b>

<b>Acknowledgements</b>: Parts of this notebook borrow heavily from examples on the [xgcm website](https://xgcm.readthedocs.io/en/latest/index.html)

---

## 1. More practice with xgcm Grid objects
In this section you'll create your own xgcm grid object and use it to compute the derivative of some data that you'll create below.

### 1.1 Create an xarray dataset with the following 1D fields:

1. $f_{1}(x_{i})=sin(x_{i})$
2. $f_{2}(x_{j})=cos(x_{j})$

Where $x_{i}\in[0,2\pi)$, $dx_{i}=\frac{\pi}{12}$, and $x_{j}=x_{i}-\frac{dx_{i}}{2}$

Make sure to add $x_{i}$ and $x_{j}$ as coordinates to your dataset.

### 1.2 Add a dx metric
Add a dx metric to your dataset that is defined at each $x_{i}$. Assume your data are periodic in x.

### 1.3 Create an xgcm grid
Create an xgcm grid object that keeps track of the relationship between $x_{i}$ and $x_{j}$. Make sure to add your dx metric as well.

### 1.4 Compute a derivative
Use the [derivative](https://xgcm.readthedocs.io/en/latest/api.html#xgcm.Grid.derivative) function to compute $\frac{df_{1}}{dx}$. 

<b>_Before_</b> you try to do the derivative, which coordinate do you expect your result to have? Do you have all the appropriate grid information (i.e., the right metrics) to compute this particular derivative?

_**After**_ you do the derivative: what do you think the warning in the output means?

### 1.5 Compute dx at $x_{j}$ points
If you keep getting this warning when you calculate the derivative, `UserWarning: Metric at ('xj',) being interpolated from metrics at dimensions ('xi',)"`, this is because your dx is defined at your $x_{i}$ points, but that's not exactly the dx that the derivative needs. We need dx defined on $x_{j}$ points instead (if this is confusing to you, it is time to draw a picture!). Thankfully, xgcm is generally smart enough to figure this out and just interpolate your grid $dx_{i}$ grid metric, which is why the computation above does not fail outright.

Regardless, go through the process of creating $dx_{j}$ by computing dx at $x_{j}$ points

><b><font color='green'>Hint:</font></b> Can you think of another xgcm grid operation that you can use that will allow you to compute dx at $x_{j}$ points? 

### 1.6 Take the derivative again
First, add your new dx at $x_{j}$ points ($dx_{j}$) to your dataset. Then recreate your xgcm grid from 1.3 but this time include your new metric.

Finally, compute $\frac{df_{1}}{dx}$, and check the coordinates that the result is on

### 1.7 Plot your result
Make a line plot showing $\frac{df_{1}}{dx}$ and $f_{2}(x_{j})$. Do your results look how you would expect?

---

## 2. Climate model output
In this section you'll be working with the CESM-LE output (`cesmLE.001.ocn.210012.nc`) we used in lecture on Monday. Go ahead and read in that data again (it's also in `/share/Notebook_data/`):

### 2.1 Create an xgcm grid for the output
Recreate the xgcm grid we built for this dataset in class on Monday. Be sure to include the metrics. You can copy and paste the code from the lecture.

### 2.2 Compute the barotropic streamfunction for the Pacific Ocean
The horizontal barotropic streamfunction $\psi$ is 

$\psi=\int_{x}\int_{z}vdzdx$

To compute $\psi$, you will need to integrate in z (the full depth of the ocean) and cumulatively integrate in x (the range of longitudes across the Pacific). Use the `REGION_MASK` variable in the dataset to pick out only the meridional velocity values in the Pacific Ocean before you integrate, otherwise you will end up doing a global cumulative integral, which isn't what we want.

You can use xgcm's [cumint()](https://xgcm.readthedocs.io/en/latest/api.html#xgcm.Grid.cumint) function for the cumulative integral. 

How should you handle the non-periodic boundary in y (i.e., what boundary keyword do you think you should use in your integral)?

### 2.3 Interpolate your result
Interpolate your streamfunction so that both x and y are located at the velocity points (xu, yu). Print the dimensions of $\psi$ beforehand to figure out how you need to interpolate your result.

### 2.4 Plot the result
Make a quick 2D contour or pcolormesh plot of your interpolated psi to visualize your results. Before you do the plotting, you'll want to mask everything outside the Pacific again and add the land back in so it's easier to tell what you're looking at. I've set up the code to do so below. In my code snippet, `psi_interp` is the interpolated streamfunction from 2.3

In [ ]:
#apply mask to the interpolated streamfunction from 2.3
psi_interp=psi_interp.where((ds.REGION_MASK==2),other=0) #pick out only pacific values, set everything outside to zero
psi_interp=psi_interp.where(ds.REGION_MASK!=0) #pick out everything but land

---